In [1]:
####### backdoor resnet18 vs normal
#### 
import torch
import sys
# sys.path.append("..")
sys.path.append("/home/yifan/projects/cophi/ContraVis")
import numpy as np

TAR_PATH = "/home/yifan/experiments/backdoor/resnet18_CIFAR10/experiment10"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"

########## initulize reference data and target data
from singleVis.DataInit import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
DEVICE = "cuda:1"
tar_datainit = DataInit(TAR_PATH,TAR_PATH,TAR_EPOCH,DEVICE)
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH,DEVICE)

ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 452.76it/s]


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 7593.97it/s]


In [2]:
####### 
import os
import json
with open(os.path.join(TAR_PATH, 'noisy_label.json')) as f:
    noise_label = json.load(f)

with open(os.path.join(TAR_PATH, 'clean_label.json'), "r") as f:
    clean_label = json.load(f)

In [3]:
noise_indicates = []
tar_train_label = tar_provider.train_labels(TAR_EPOCH)
for i in range(len(noise_label)):
    if noise_label[i] != tar_train_label[i]:
        noise_indicates.append(i)


In [9]:
train_labels = tar_provider.train_labels(TAR_EPOCH)
tar_data = tar_provider.train_representation(TAR_EPOCH)
tar_data = tar_data.reshape(tar_data.shape[0],tar_data.shape[1])
tar_pred = tar_provider.get_pred(TAR_EPOCH, tar_data)
tar_pred = tar_pred.argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 6457.94it/s]


In [10]:

k = 0
for i in range(len(tar_pred)):
    if tar_pred[i] != train_labels[i]:
        k = k+1
print(k)

13


In [13]:
from contrast.transfomration import TransformationTrainer

trainer = TransformationTrainer(ref_train_data,tar_train_data, DEVICE)
tarns_model,tar_mapped,ref_reconstructed  = trainer.transformation_train( num_epochs= 200)
# tarns_model,tar_mapped,ref_reconstructed  = trainer.transformation_train()

Epoch [1/200], Loss: 0.6939
Epoch [21/200], Loss: 0.3620
Epoch [41/200], Loss: 0.2748
Epoch [61/200], Loss: 0.2020
Epoch [81/200], Loss: 0.1716
Epoch [101/200], Loss: 0.1654
Epoch [121/200], Loss: 0.1636
Epoch [141/200], Loss: 0.1624
Epoch [161/200], Loss: 0.1614
Epoch [181/200], Loss: 0.1604


In [14]:

ref_pred = ref_provider.get_pred(REF_EPOCH, ref_train_data).argmax(axis=1)
tar_pred = tar_provider.get_pred(TAR_EPOCH, tar_train_data).argmax(axis=1)
reconstructed_pred = tar_provider.get_pred(TAR_EPOCH, ref_reconstructed).argmax(axis=1)
diff = 0
for i in range(len(reconstructed_pred)):
    if reconstructed_pred[i] != tar_pred[i]:
        diff = diff+1
print("reconstruct prediction different number:",diff)

100%|██████████| 250/250 [00:00<00:00, 5477.54it/s]


reconstruct prediction different number: 3


In [15]:
tar_train_label

array([6, 9, 9, ..., 9, 1, 1])